In [ ]:
import geopandas as gpd
import pandas as pd
import pickle
import momepy
import dask
from dask.distributed import Client, LocalCluster
from tqdm.auto import tqdm
from scripts.percentile import Percentiles


In [ ]:
local_crs = 3414

place = "singapore"

In [ ]:
daskCluster = LocalCluster(threads_per_worker=2,
                n_workers=8, memory_limit='100GB')

client = Client(daskCluster, timeout="300s")

client



In [ ]:
tessellation = gpd.read_parquet(f"./out/{place}/tessellation_stats.pq")
buildings = gpd.read_parquet(f"./out/{place}/buildings.pq")

with open(f"./out/{place}/weights.pickle", 'rb') as f:
    queen_1 = pickle.load(f)
    queen_3 = pickle.load(f)
    dist200 = pickle.load(f)
    f.close()
    

In [ ]:
percentiles = []
for column in tqdm(tessellation.columns.drop(["geometry", "uID"]), total = len(tessellation.columns.drop(["geometry", "uID"]))):
    perc = Percentiles(tessellation, column, queen_3, "uID", verbose=True, client=client).frame
    perc.columns = [f"{column}_" + str(x) for x in perc.columns]
    percentiles.append(perc)
    
percentiles.append(tessellation[["uID", "geometry"]])

In [ ]:
percentiles_joined = pd.concat(percentiles, axis=1)
percentiles_joined.head()

In [ ]:
percentiles_joined = gpd.GeoDataFrame(percentiles_joined, geometry="geometry", crs=local_crs)

In [ ]:
print(percentiles_joined.columns.tolist())

In [ ]:
percentiles_joined.to_parquet(f"./out/{place}/tessellation_percentiles.pq")